# Lab 3.1.5: 70B Model QLoRA Fine-Tuning ⭐

**Module:** 3.1 - Large Language Model Fine-Tuning  
**Time:** 4 hours  
**Difficulty:** ⭐⭐⭐⭐☆

---

## 🚀 THIS IS THE DGX SPARK SHOWCASE!

**What you're about to do would be impossible on any consumer GPU:**

| Hardware | 70B Fine-Tuning? | Cost |
|----------|-----------------|------|
| RTX 4090 (24GB) | ❌ Impossible | N/A |
| 2× RTX 4090 (48GB) | ❌ Still impossible | ~$4,000 |
| 4× A100 80GB (Cloud) | ✅ Works | ~$20/hour |
| **DGX Spark (128GB)** | ✅ **Works!** | **One-time purchase** |

**With 128GB unified memory, you can fine-tune Llama 3.1 70B on your DESKTOP!**

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Fine-tune a 70B parameter model on DGX Spark
- [ ] Understand memory optimization techniques for large models
- [ ] Document memory usage patterns for 70B training
- [ ] Compare 70B vs 8B fine-tuning quality

---

## Prerequisites

- Completed: Lab 3.1.4 (8B Model Fine-Tuning)
- **Model access required:** Request access to `Qwen/Qwen3-32B-Instruct` on HuggingFace
- **Time:** First run will download ~40GB of model weights

---

## ELI5: Why is This Special?

> **Imagine you have a giant library with 70 billion books.** Normally, you'd need a warehouse (cloud GPUs) to store and work with all these books.
>
> **DGX Spark's 128GB unified memory is like having a really efficient librarian** who can compress those books (4-bit quantization) while still being able to read them. The books take up 4× less space, and you can work with ALL of them on your desk!
>
> **The magic:**
> - 70B parameters × 2 bytes (FP16) = 140GB (won't fit)
> - 70B parameters × 0.5 bytes (4-bit) = **~35GB** (fits easily!)
> - Add LoRA adapters + training state = **~45-55GB total**
> - That leaves **70GB+ for batch data and overhead!**

---

## Part 1: Pre-Flight Checks

Before loading a 70B model, we need to ensure optimal conditions.

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
# ║ CRITICAL: Clear system caches before loading 70B model                      ║
# ║ This frees up memory fragmentation from previous operations                 ║
# ╚════════════════════════════════════════════════════════════════════════════╝

import subprocess
import os

# Clear Linux buffer cache (requires sudo)
# This is ESSENTIAL for reliable 70B loading
print("Clearing system caches...")
try:
    result = subprocess.run(
        ["sudo", "sh", "-c", "sync; echo 3 > /proc/sys/vm/drop_caches"],
        capture_output=True, text=True, timeout=30
    )
    if result.returncode == 0:
        print("✅ System caches cleared successfully")
    else:
        print(f"⚠️ Could not clear caches: {result.stderr}")
        print("   You may need to run: sudo sh -c 'sync; echo 3 > /proc/sys/vm/drop_caches'")
except Exception as e:
    print(f"⚠️ Cache clearing skipped: {e}")
    print("   Consider running manually if you encounter OOM issues")

In [ ]:
import torch
import gc
import psutil
from typing import Dict

def get_system_memory() -> Dict[str, float]:
    """Get comprehensive memory statistics."""
    stats = {}
    
    # GPU memory
    if torch.cuda.is_available():
        stats['gpu_allocated'] = torch.cuda.memory_allocated() / 1e9
        stats['gpu_reserved'] = torch.cuda.memory_reserved() / 1e9
        stats['gpu_total'] = torch.cuda.get_device_properties(0).total_memory / 1e9
        stats['gpu_free'] = stats['gpu_total'] - stats['gpu_allocated']
    
    # System memory (for unified memory awareness)
    mem = psutil.virtual_memory()
    stats['system_total'] = mem.total / 1e9
    stats['system_available'] = mem.available / 1e9
    stats['system_used'] = mem.used / 1e9
    
    return stats

def print_memory_status():
    """Print formatted memory status."""
    stats = get_system_memory()
    print("\n" + "="*60)
    print("MEMORY STATUS")
    print("="*60)
    
    if 'gpu_total' in stats:
        print(f"\nGPU Memory:")
        print(f"  Total:     {stats['gpu_total']:.1f} GB")
        print(f"  Allocated: {stats['gpu_allocated']:.1f} GB")
        print(f"  Free:      {stats['gpu_free']:.1f} GB")
        
        # DGX Spark detection
        if stats['gpu_total'] > 100:
            print(f"\n🚀 DGX Spark detected! 128GB unified memory available.")
            print(f"   You have {stats['gpu_free']:.1f} GB free for the 70B model!")
    
    print(f"\nSystem Memory:")
    print(f"  Total:     {stats['system_total']:.1f} GB")
    print(f"  Available: {stats['system_available']:.1f} GB")
    print(f"  Used:      {stats['system_used']:.1f} GB")
    print("="*60)

print_memory_status()

In [ ]:
# Verify we have enough memory
stats = get_system_memory()

REQUIRED_MEMORY_GB = 60  # Conservative estimate for 70B training

if stats.get('gpu_free', 0) < REQUIRED_MEMORY_GB:
    print(f"⚠️ WARNING: Only {stats.get('gpu_free', 0):.1f} GB free.")
    print(f"   Recommended: {REQUIRED_MEMORY_GB} GB for comfortable 70B training.")
    print("\n   Options:")
    print("   1. Clear buffer cache: sudo sh -c 'sync; echo 3 > /proc/sys/vm/drop_caches'")
    print("   2. Restart Jupyter kernel")
    print("   3. Close other GPU-using applications")
else:
    print(f"✅ Sufficient memory available: {stats.get('gpu_free', 0):.1f} GB")
    print(f"   Ready to load 70B model!")

---

## Part 2: Configuration

Optimized settings for 70B model training on DGX Spark.

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
# ║                         70B TRAINING CONFIGURATION                          ║
# ╚════════════════════════════════════════════════════════════════════════════╝

# Model selection
MODEL_NAME = "Qwen/Qwen3-32B-Instruct"  # The big one!
# Alternative if you don't have Llama access:
# MODEL_NAME = "Qwen/Qwen2-72B-Instruct"  # Another 70B+ option

# Output
OUTPUT_DIR = "./finetuned-70b-model"

# Sequence length - keep shorter for 70B to save memory
MAX_SEQ_LENGTH = 512  # Reduced from 1024 for memory efficiency

# Batch size - be conservative with 70B
BATCH_SIZE = 1  # Start with 1, can increase if memory allows
GRADIENT_ACCUMULATION = 16  # Compensate with gradient accumulation

# Training schedule
LEARNING_RATE = 1e-4  # Slightly lower for stability
NUM_EPOCHS = 1
WARMUP_RATIO = 0.1

# LoRA configuration - more conservative for 70B
LORA_RANK = 16  # Can go higher if memory allows
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
USE_DORA = True  # Still use DoRA for quality

# NEFTune
NEFTUNE_ALPHA = 5.0

# Target modules - attention only for 70B to save memory
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]
# Add MLP if you have memory headroom:
# TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

print("70B Training Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  LoRA rank: {LORA_RANK} (alpha={LORA_ALPHA})")
print(f"  DoRA: {'Enabled' if USE_DORA else 'Disabled'}")
print(f"  NEFTune: α={NEFTUNE_ALPHA}")
print(f"  Batch size: {BATCH_SIZE} × {GRADIENT_ACCUMULATION} = {BATCH_SIZE * GRADIENT_ACCUMULATION} effective")
print(f"  Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  Target modules: {TARGET_MODULES}")

---

## Part 3: Load the 70B Model

This is where the magic happens. We'll load 70 billion parameters into memory!

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

print("Libraries imported successfully!")

In [ ]:
# QLoRA Configuration - optimized for 70B
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,  # Double quantization saves ~5GB
)

print("Quantization config ready:")
print("  4-bit NF4 quantization")
print("  Double quantization enabled")
print("  Compute dtype: bfloat16")

In [ ]:
# Load tokenizer first (lightweight)
print(f"Loading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✅ Tokenizer loaded (vocab size: {tokenizer.vocab_size:,})")

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
# ║ LOADING 70B MODEL - This is the big moment!                                 ║
# ║ First run: Downloads ~40GB. Subsequent runs: Loads from cache.              ║
# ╚════════════════════════════════════════════════════════════════════════════╝

import time

print(f"\n{'='*60}")
print(f"LOADING {MODEL_NAME}")
print(f"This is 70 BILLION parameters!")
print(f"{'='*60}")

print(f"\nMemory before loading:")
print_memory_status()

start_time = time.time()

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="sdpa",  # Efficient attention
    torch_dtype=torch.bfloat16,
)

load_time = time.time() - start_time

print(f"\n{'='*60}")
print(f"✅ 70B MODEL LOADED SUCCESSFULLY!")
print(f"   Load time: {load_time:.1f} seconds ({load_time/60:.1f} minutes)")
print(f"{'='*60}")

print(f"\nMemory after loading:")
print_memory_status()

In [ ]:
# Prepare for training
print("Preparing model for k-bit training...")

model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=True,  # ESSENTIAL for 70B
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

print("✅ Model prepared with gradient checkpointing enabled")

In [ ]:
# Apply LoRA/DoRA
lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=USE_DORA,
)

model = get_peft_model(model, lora_config)

print(f"\n{'='*60}")
print(f"{'DoRA' if USE_DORA else 'LoRA'} Adapters Added!")
print(f"{'='*60}")
model.print_trainable_parameters()

print(f"\nMemory after adding adapters:")
print_memory_status()

### 🎉 You Just Loaded a 70B Model on Your Desktop!

Take a moment to appreciate this:
- **70 billion parameters** compressed to ~35-40GB
- **Still fits** with room for training
- **No cloud required** - this is YOUR computer!

---

## Part 4: Prepare Dataset

In [ ]:
# Load dataset (using smaller subset for 70B to keep training reasonable)
print("Loading dataset...")
dataset = load_dataset("tatsu-lab/alpaca", split="train[:500]")

print(f"Dataset: {len(dataset)} examples")

In [ ]:
# Format for Llama 3.1
def format_instruction_llama3(example):
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']
    
    if input_text:
        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{instruction}

Input: {input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output}<|eot_id|>"""
    else:
        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output}<|eot_id|>"""
    
    return {"text": prompt}

formatted_dataset = dataset.map(format_instruction_llama3)
print(f"Dataset formatted!")

---

## Part 5: Training Configuration

Optimized settings for 70B on DGX Spark.

In [ ]:
# Training arguments - optimized for 70B on DGX Spark
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    
    # Conservative batch size for 70B
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    
    # Training schedule
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,
    
    # Optimization
    optim="paged_adamw_8bit",  # Memory-efficient
    weight_decay=0.01,
    max_grad_norm=0.3,  # Lower for stability with large models
    
    # Precision
    bf16=True,
    
    # Sequence length
    max_seq_length=MAX_SEQ_LENGTH,
    packing=True,
    
    # NEFTune
    neftune_noise_alpha=NEFTUNE_ALPHA,
    
    # Logging
    logging_steps=5,
    logging_first_step=True,
    
    # Saving
    save_strategy="epoch",
    save_total_limit=1,  # Save space
    
    # Performance
    gradient_checkpointing=True,
    
    report_to="none",
    seed=42,
)

print("70B Training Configuration Ready!")

In [ ]:
# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
)

print("Trainer created!")
print_memory_status()

---

## Part 6: Train the 70B Model!

This is it - training 70 billion parameters on your desktop!

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
# ║                    TRAINING 70B MODEL ON DGX SPARK                          ║
# ╠════════════════════════════════════════════════════════════════════════════╣
# ║ This would be IMPOSSIBLE on any consumer GPU!                               ║
# ║ You're about to train one of the largest open-source models on your desk!  ║
# ╚════════════════════════════════════════════════════════════════════════════╝

print("\n" + "🔥"*30)
print("STARTING 70B MODEL TRAINING!")
print("🔥"*30 + "\n")

print(f"Model: {MODEL_NAME}")
print(f"Parameters: 70 BILLION")
print(f"Method: {'DoRA' if USE_DORA else 'LoRA'} + NEFTune")
print(f"Dataset: {len(formatted_dataset)} examples")
print(f"\nThis is running on YOUR DGX Spark!")

# Memory status before training
print_memory_status()

print("\n" + "="*60)
print("Training starting...")
print("="*60 + "\n")

start_time = time.time()

try:
    train_result = trainer.train()
    training_time = time.time() - start_time
    
    print("\n" + "🎉"*30)
    print("70B TRAINING COMPLETE!")
    print("🎉"*30)
    print(f"\nTraining time: {training_time:.1f} seconds ({training_time/60:.1f} minutes)")
    
except Exception as e:
    print(f"\n❌ Training error: {e}")
    print("\nTroubleshooting:")
    print("  1. Try reducing MAX_SEQ_LENGTH to 256")
    print("  2. Try reducing LORA_RANK to 8")
    print("  3. Clear caches and restart kernel")
    raise

In [ ]:
# Print training metrics
print("\nTraining Metrics:")
for key, value in train_result.metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

print_memory_status()

---

## Part 7: Test Your 70B Model

In [ ]:
def generate_response_70b(prompt: str, max_new_tokens: int = 256) -> str:
    """Generate response from fine-tuned 70B model."""
    formatted = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract assistant response
    if "assistant" in response.lower():
        parts = response.split("assistant")
        response = parts[-1].strip() if len(parts) > 1 else response
    
    return response

In [ ]:
# Test the 70B model
test_prompts = [
    "Explain quantum computing to a 10-year-old.",
    "What are the main differences between Python and JavaScript?",
    "Write a haiku about artificial intelligence.",
]

print("Testing fine-tuned 70B model...")
print("="*60)

for prompt in test_prompts:
    print(f"\n📝 Prompt: {prompt}")
    print("-"*40)
    response = generate_response_70b(prompt)
    print(f"🤖 Response: {response[:500]}..." if len(response) > 500 else f"🤖 Response: {response}")
    print("="*60)

---

## Part 8: Save Your 70B Adapters

In [ ]:
# Save the LoRA adapters
adapter_path = os.path.join(OUTPUT_DIR, "70b_adapter")
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)

print(f"✅ 70B Adapter saved to: {adapter_path}")

# Show saved files
print("\nSaved files:")
total_size = 0
for f in os.listdir(adapter_path):
    size = os.path.getsize(os.path.join(adapter_path, f)) / 1e6
    total_size += size
    print(f"  {f}: {size:.2f} MB")

print(f"\nTotal adapter size: {total_size:.2f} MB")
print(f"(Compare to full 70B model: ~140GB in FP16!)")

---

## Part 9: Memory Analysis Report

In [ ]:
# Generate memory report
print("\n" + "="*60)
print("70B TRAINING MEMORY ANALYSIS")
print("="*60)

final_stats = get_system_memory()

print(f"""
SUMMARY
-------
Model: {MODEL_NAME}
Parameters: 70,000,000,000 (70B)
Quantization: 4-bit NF4 with double quantization

Memory Usage:
  Base model (4-bit):     ~35-40 GB
  LoRA adapters:          ~{(LORA_RANK * 2 * 4096 * len(TARGET_MODULES) * 80) / 1e6:.0f} MB
  Optimizer states:       ~2-3 GB
  Gradients + activations: ~5-10 GB
  ----------------------------------
  Peak training memory:   ~45-55 GB

DGX Spark Capacity:
  Total unified memory:   128 GB
  Used for training:      ~{final_stats.get('gpu_allocated', 0):.1f} GB
  Remaining headroom:     ~{128 - final_stats.get('gpu_allocated', 50):.1f} GB

What This Means:
  - RTX 4090 (24GB): ❌ IMPOSSIBLE
  - 2× RTX 4090 (48GB): ❌ IMPOSSIBLE  
  - A100 40GB: ❌ IMPOSSIBLE
  - A100 80GB (cloud): ⚠️ Tight, needs careful tuning
  - DGX Spark (128GB): ✅ COMFORTABLE with headroom!
""")

print("="*60)

---

## Summary: What You Just Accomplished

### 🏆 Achievement Unlocked: 70B Fine-Tuning Master!

You successfully:

1. ✅ Loaded a **70 billion parameter model** on your desktop
2. ✅ Applied **QLoRA** (4-bit quantization) for memory efficiency
3. ✅ Added **DoRA** adapters for quality improvement
4. ✅ Enabled **NEFTune** for better generalization
5. ✅ **Trained** the model on instruction data
6. ✅ **Tested** and saved your custom 70B model

### DGX Spark vs. Alternatives

| Approach | Hardware | Cost | 70B Training? |
|----------|----------|------|---------------|
| DGX Spark | 128GB unified | One-time purchase | ✅ Yes! |
| Cloud A100s | 4× 80GB | ~$80/hour | ✅ Yes |
| RTX 4090s | 2× 24GB | ~$4,000 | ❌ No |

**You just did in your home office what normally requires cloud infrastructure!**

---

## Cleanup

In [ ]:
# Clear GPU memory
print("Cleaning up...")

del model, trainer
torch.cuda.empty_cache()
gc.collect()

print("✅ Cleanup complete!")
print_memory_status()

---

## Next Steps

Congratulations! You've mastered the core fine-tuning skills. Continue to:

**[Lab 3.1.6: Dataset Preparation](lab-3.1.6-dataset-preparation.ipynb)** - Learn to create your own high-quality training datasets!